<a href="https://colab.research.google.com/github/Collectorr1988/Portfolio/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Giga_chat_%D0%A5%D0%B0%D0%BA%D0%B0%D1%82%D0%BE%D0%BD_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gigachain

In [ ]:
pip install gigachat

In [ ]:
%pip install chromadb

In [ ]:
pip install pypdf

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings import GigaChatEmbeddings
from langchain.chat_models.gigachat import GigaChat
from langchain.schema import HumanMessage
from langchain.chains import RetrievalQA
from pypdf import PdfReader


In [24]:
loader = PyPDFLoader("1.pdf") #файл прикладываю к решению
documents = loader.load()

PdfStreamError: Stream has ended unexpectedly

In [ ]:
documents

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=800,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

In [10]:
embeddings = GigaChatEmbeddings(
    credentials='NTlkY2MyZmItM2Q4ZC00ZWMzLWE2NjAtNTI3MzZhOTk2ZjQzOjVhZGJiZDQxLTc0YjAtNDQxNi04YjAzLTUxZDVmYTY4NTkwNw==', scope='GIGACHAT_API_CORP',
                verify_ssl_certs=False)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False),
)

In [11]:
llm = GigaChat(temperature=0.0,
    credentials='NTlkY2MyZmItM2Q4ZC00ZWMzLWE2NjAtNTI3MzZhOTk2ZjQzOjVhZGJiZDQxLTc0YjAtNDQxNi04YjAzLTUxZDVmYTY4NTkwNw==',
               scope='GIGACHAT_API_CORP',
               chain_type="stuff",
                verify_ssl_certs=False)

In [ ]:
question = "Show all compponents of red mud in the table "
llm([HumanMessage(content=question)]).content[0:800]

In [ ]:
docs = db.similarity_search(question, k=10)
len(docs)

In [14]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [ ]:
display(qa_chain)

In [16]:
prompt = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: Composition wt %Fe2O3 42.34, Al2O3 16.26, Ignition loss 12.66, CaO 11.64, SiO2


Question: Show full chemichal composition of red mud



Answer: """

In [17]:
composition=qa_chain({"query": prompt})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [18]:
composition

{'query': "Ответь на вопрос, опираясь на контекст ниже.\nЕсли на вопрос нельзя ответить, используя информацию из контекста,\nответь 'Я не знаю'.\n\nContext: Composition wt %Fe2O3 42.34, Al2O3 16.26, Ignition loss 12.66, CaO 11.64, SiO2 \n\n\nQuestion: Show full chemichal composition of red mud\n\n\n\nAnswer: ",
 'result': 'Table 1. Chemical composition of red mud.\nComposition wt %\nFe2O3 42.34\nAl2O3 16.26\nIgnition loss 12.66\nCaO 11.64\nSiO 2 6.97\nTiO 2 4.27\nNa2O 3.83\nOthers 1.85\nLa2O3 0.09\nCeO 2 0.06\nSc2O3 0.02\nNd2O3 0.01\nY2O3 0.01'}

In [19]:
print(composition['result'])

Table 1. Chemical composition of red mud.
Composition wt %
Fe2O3 42.34
Al2O3 16.26
Ignition loss 12.66
CaO 11.64
SiO 2 6.97
TiO 2 4.27
Na2O 3.83
Others 1.85
La2O3 0.09
CeO 2 0.06
Sc2O3 0.02
Nd2O3 0.01
Y2O3 0.01


In [20]:
oxide=composition['result']

In [21]:
import pandas as pd

# Создаем список данных
data = [['referense_1', oxide]]

# Создаем DataFrame из списка данных
df = pd.DataFrame(data, columns=['Reference', 'Chemichal composition of red mud'])

# Выводим DataFrame в столбец
print(df.to_string(index=False))

  Reference                                                                                                                                                                                                 Chemichal composition of red mud
referense_1 Table 1. Chemical composition of red mud.\nComposition wt %\nFe2O3 42.34\nAl2O3 16.26\nIgnition loss 12.66\nCaO 11.64\nSiO 2 6.97\nTiO 2 4.27\nNa2O 3.83\nOthers 1.85\nLa2O3 0.09\nCeO 2 0.06\nSc2O3 0.02\nNd2O3 0.01\nY2O3 0.01


In [22]:
display(df)

,Reference,Chemichal composition of red mud
0,referense_1,Table 1. Chemical composition of red mud.\nCom...


Смысл работы заключается в том, чтобы создать программу которая будет автоматический вытаскивать из статей химические составы материаллов в таблицу и указывать источник. Учитывая, что таких статей тысячи это долго делать вручную. дальше планируются сделать столбец "эксперимент", чтобы уазать как эти материаллы были обработаны и столбец "результат", чтобы указать составы которые получились. Это очень поможет в работе химикам.